# 1293. Shortest Path in a Grid with Obstacles Elimination

## Topic Alignment
- State-augmented BFS models scenarios where planners may consume limited budget to bypass obstacles, analogous to resource-constrained routing in robotics or feature toggling during automated experimentation.


## Metadata 摘要
- Source: https://leetcode.com/problems/shortest-path-in-a-grid-with-obstacles-elimination/
- Tags: BFS, Grid, State Search
- Difficulty: Hard
- Priority: High

## Problem Statement 原题描述
You are given an m x n grid with obstacles (1) and empty cells (0), plus an integer k representing the maximum number of obstacles you can eliminate. Return the minimum number of steps to move from (0,0) to (m-1,n-1) with at most k eliminations, or -1 if impossible.

## Progressive Hints
- Hint 1: Encode the state as `(row, col, remaining_eliminations)`.
- Hint 2: Keep a best-remaining-k record per cell so you do not revisit it with an inferior budget.
- Hint 3: Early exit once you reach the bottom-right corner; BFS guarantees it is optimal.


## Solution Overview
This is a shortest-path problem on an implicit graph whose nodes store both a grid location and how many obstacles can still be removed. Run BFS starting from `(0, 0, k)`. When expanding to a neighbor containing a `1`, consume one elimination. If the new state improves the remaining budget recorded for that cell, push it into the queue. The first time we poll `(m-1, n-1)` we have found the shortest path length.


## Detailed Explanation
1. Handle the trivial case `m == 1` and `n == 1` by returning 0.
2. If `k` is large enough to bypass every obstacle along the Manhattan path (`k >= m + n - 2`), return that distance directly.
3. Initialize a queue with `(0, 0, k)` and a `visited` matrix that stores the best remaining budget seen for each cell (initialize to `-1`).
4. While the queue is non-empty:
   - For the current BFS level, iterate over all states.
   - When popping `(r, c, remaining)`, return the steps if `(r, c)` is the target.
   - Otherwise, iterate the four directional moves; compute `(nr, nc)` and the new budget `nk = remaining - grid[nr][nc]`.
   - Skip invalid coordinates or states where `nk < 0` or `visited[nr][nc] >= nk`.
   - Update `visited[nr][nc] = nk` and enqueue `(nr, nc, nk)`.
5. Increase the step counter after processing one level.
6. Return `-1` if the queue empties without reaching the target.


## Complexity Trade-off Table
| Approach | Time | Space | Notes |
| --- | --- | --- | --- |
| BFS with state `(r, c, k)` | O(m * n * k) | O(m * n) | Visits each cell for each feasible remaining budget |
| Dijkstra on weighted graph | O(m * n * k log(mnk)) | O(m * n * k) | Overkill because weights are only 0 or 1 |
| DFS with pruning | Exponential | O(m * n) | Cannot guarantee optimality; likely to TLE |


In [ ]:
from collections import deque
from typing import List

class Solution:
    def shortestPath(self, grid: List[List[int]], k: int) -> int:
        rows, cols = len(grid), len(grid[0])
        if rows == 1 and cols == 1:
            return 0
        if k >= rows + cols - 2:
            return rows + cols - 2
        dirs = [(1,0),(-1,0),(0,1),(0,-1)]
        visited = [[-1] * cols for _ in range(rows)]
        queue = deque([(0, 0, k)])
        visited[0][0] = k
        steps = 0
        while queue:
            for _ in range(len(queue)):
                r, c, remain = queue.popleft()
                if r == rows - 1 and c == cols - 1:
                    return steps
                for dr, dc in dirs:
                    nr, nc = r + dr, c + dc
                    if 0 <= nr < rows and 0 <= nc < cols:
                        nk = remain - grid[nr][nc]
                        if nk < 0:
                            continue
                        if visited[nr][nc] >= nk:
                            continue
                        visited[nr][nc] = nk
                        queue.append((nr, nc, nk))
            steps += 1
        return -1


In [ ]:
tests = [
    (([[0,0,0],[1,1,0],[0,0,0],[0,1,1],[0,0,0]], 1), 6),
    (([[0,1,1],[1,1,1],[1,0,0]], 1), -1),
    (([[0,0],[0,0]], 0), 2)
]
solver = Solution()
for (grid, k), expected in tests:
    grid_copy = [row[:] for row in grid]
    assert solver.shortestPath(grid_copy, k) == expected
print('All tests passed.')


## Complexity Analysis
- Time: O(m * n * k) since each pair `(cell, remaining)` is processed at most once.
- Space: O(m * n) for the queue and the `visited` budget table.


## Edge Cases & Pitfalls
- Handle the case where `k` already exceeds the Manhattan distance—no BFS needed.
- Do not revisit a cell with the same or smaller remaining budget; it would create redundant work.
- Copy the grid before mutating it if the caller expects the original grid to remain unchanged (tests here clone rows).


## Follow-up Variants
- Return the actual path by storing predecessors together with each state.
- Allow different costs for removing obstacles; convert to Dijkstra with weighted edges.
- Support multiple agents with shared elimination budget by extending the state to include agent identifiers.


## Takeaways
- Augmenting BFS state with remaining resources is a common pattern for constrained shortest-path problems.
- Tracking the best budget seen per cell is analogous to dynamic programming and keeps the search polynomial.
- Recognizing when `k` trivially satisfies the Manhattan distance yields a quick optimization.


## Similar Problems
| Problem ID | Problem Title | Technique |
| --- | --- | --- |
| LC 1091 | Shortest Path in Binary Matrix | BFS with 8-direction movement |
| LC 864 | Shortest Path to Get All Keys | BFS with state augmentation for collected keys |
| LC 499 | The Maze III | BFS/priority search with stateful positions |
